# Market-Basket Analysis-Frequent Itemsets Generation

In this notebook we perform Market-Basket Analysis by generating **frequent itemsets**.

We implement a **brute-force** algorithm for generating frequent 2-itemsets.

Our goal is to understand the time-complexity of the brute-force technique.


## Transaction Data Format

The Market-Basket data is typically given using a Python **list of lists** structure.

For the purpose of programmatic processing by the standard APIs (e.g., Mlxtend or machine learning extensions API), the Market-Basket data needs to be stored in a **binary format**. 

More specifically, in a transaction an item can be treated as a binary variable whose value is 1 if the item is present in a transaction and 0 otherwise. This is known as **one-hot encoding**.



## Input Data

The transaction data is stored in a CSV file.

Each row contains a comma separated list of items.

Thus, to create one-hot encoded data we do the following:
- Read from the CSV file as a python **list of lists**. Remove white spaces (empty items).
- One-hot encode the data from list of lists 

The brute-force function takes this one-hot encoded data as input and creates the following:
- A list of unique items
- A list of transactions in which every transaction (set of items purchased by a customer) is associated with a transaction id. We design this data structure as a Python dictionary.

Thus, using the one-encoded data we will have to create a list of items and  dictionary of transaction.


### Create One-Hot Encoded Data
To create a one-hot encoded NumPy boolean array of the transaction data from a Python list of lists, we use the **TransactionEncoder** object from the mlextend API.

The TransactionEncoder uses its "fit" method to learn the unique labels in the dataset, and the "transform" method to transform the input dataset (a Python list of lists) into a one-hot encoded NumPy boolean array.

In [1]:
from time import time

import numpy as np
import pandas as pd
import csv

from mlxtend.preprocessing import TransactionEncoder

## Helper Functions for Data Pre-Processing


We use several helper functions for data pre-processing.

- isInList(): Function to determine whether an item/itemset is contained in a list/basket.

- removeEmptyValues(): Function to remove strings (e.g., white space) from the list items.

- generateListOfListsFromCSV(): Function to create a **list of lists** from a CSV file containing transaction data.

- oneHotEncodedDataFrame(): Function to create a one-hot encoded DataFrame object from the Python **list of lists**. 

- generateItemsBaskets(): Function to create a list of items & baskets from a one-hot encoded DataFrame object. This function is required by the brute-force frequent itemsets generation function.

In [2]:
# Function to determine whether an item/itemset is contained in a list/basket
def isInList(aList, item):
    if item in aList:
        return 1
    else:
        return -1   

    
# Function to remove strings (e.g., white space) from the list items
def removeEmptyValues(aList, symbolToRemove):
    while(True):
        val = isInList(aList, symbolToRemove)
        if(val == 1):
            aList.remove(symbolToRemove)    
        else:
            break

        

# Function to create a list of lists from a CSV file containing transaction data
def generateListOfListsFromCSV(fullFileName):

    with open(fullFileName, newline='', encoding='utf-8-sig') as f:
        reader = csv.reader(f, delimiter=",")
        data = list(reader)

    listOfLists = []

    for i in range(len(data)):
        removeEmptyValues(data[i], '')
        listOfLists.append(data[i])
        
    return listOfLists

 

# Function to create a one-hot encoded DataFrame object from the Python list of lists        
def oneHotEncodedDataFrame(data):

    # Transform the transaction data into a one-hot encoded NumPy boolean array 
    te = TransactionEncoder()
    te_ary = te.fit(data).transform(data)


    # Read the one-hot encoded data as a Pandas DataFrame object
    dataFrame = pd.DataFrame(te_ary, columns=te.columns_)
    
    return dataFrame


# Function to create a list of items & baskets from a one-hot encoded DataFrame object
def generateItemsBaskets(dataFrame):    
    # Create a list of items from the DataFrame object
    items = []
    for col in dataFrame.columns: 
        items.append(col)
        
    # Create a list of baskets from the DataFrame object
    baskets = []
    transaction_id = 1

    for i in range(len(dataFrame)):
        itemset = []
        for j in range(len(items)):
            if(dataFrame.iloc[i][j] == True):
                itemset.append(items[j])
        baskets.append({transaction_id: itemset})
        transaction_id += 1
        
    return items, baskets
        

## Load Transaction Data From CSV File

In [3]:
# Load the CSV file transaction data as a **list of lists**.
dataset = generateListOfListsFromCSV('/Users/hasan/datasets/transactionsmall.csv')

## Brute-Force Algorithm: Frequent 2-Itemsets Generation

We define a function to implement the brute-force algorithm for generating frequent 2-itemsets from a one-hot encoded DataFrame.

Following are the parameters used by the function.

- dataFrame : pandas DataFrame or pandas SparseDataFrame

The transaction dataset in pandas one-hot encoded DataFrame format. The allowed values are either 0/1 or True/False.

- min_support : int

It is the minumum support count of the itemsets returned. 

- max_len : int (default: None)

Maximum length of the itemsets generated. Currently the function does not use this parameter. The function is hard-coded to compute only frequent 2-itemsets.


#### The function returns a list of all 2-itemsets that are >= min_support.

In [4]:
def bruteForceFrequentItemsets(dataFrame, min_support=2, max_len=None):
    
    
    # Create a list of items & baskets from a one-hot encoded DataFrame object
    items, baskets = generateItemsBaskets(dataFrame)   
    
    # Declare a list to store frequent 2-itemsets
    freq_itemsets = []

    
    '''
    Brute-force Aproach:
    
    - Identify candidate 2-itemsets
        -- Go through every consequtive pairs of distinct items the transactions.
    - Identify frequent 2-itemsets:
        -- Determine whether the candidate itemsets exist in the baskets, and increment their basket count.
        -- If the basket count of an itemset is greater than or equal to the minimum support count,
     store it in the frequent itemsets list.
    
    '''
    for i in range(len(items)):

        for j in range(len(items)):

            if(j != i and j > i):
                basket_count = 0
                basket_item = 1
                for b in baskets:
                    val1 = isInList(b.get(basket_item), items[i])
                    val2 = isInList(b.get(basket_item), items[j])
                    if(val1 != -1 and val2 != -1):
                        basket_count += 1
                    basket_item += 1


                if(basket_count >= min_support):
                    freq_itemsets.append([items[i], items[j]])
                    
    return freq_itemsets
   

## Generate Frequent 2-itemsets Using Brute-force Algorithm

Previously we loaded the transaction dataset as a **list of lists** from the CSV file.

The brute-force function requires this data as a one-hot encoded Pandas DataFrame.

Thus, first we convert the **list of lists** transaction data as one-hot encoded Pandas DataFrame object. Then, pass it to the brute-force function for generating frequent 2-itemsets.

We compute the running-time of the brute-force function.

In [5]:
# Convert the **list of lists** transaction data as one-hot encoded Pandas DataFrame object.
dataset_onehotencoded = oneHotEncodedDataFrame(dataset)

t0 = time()    

# Generate frequent 2-itemsets using the brute-force function
frequent_itemsets = bruteForceFrequentItemsets(dataset_onehotencoded, min_support=3)

time_brute_force = time() - t0

print("Brute-force: Running Time = %0.3fs" % time_brute_force)

print("\n____________________")
print("Frequent 2-itemsets:")
for i in range(len(frequent_itemsets)):
    print(frequent_itemsets[i])

Brute-force: Running Time = 0.008s

____________________
Frequent 2-itemsets:
['M1', 'M2']
['M1', 'M3']
['M1', 'M4']
['M2', 'M4']
['M2', 'M5']
['M4', 'M5']
